# Part 3: GLMM Analysis


## 1. Creating the model predictors using GridFix

In this part of the tutorial, we will use the grid and image features defined in the previous chapters to create a GLMM predictor matrix and output some model code for R. First, let's reproduce the analysis from previous chapters and add some features that might influence each grid cell's fixation probability:

In [1]:
%matplotlib inline

import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt

from gridfix import *

# Load images and define 8x6 grid from part 1
TP_images = ImageSet('TP_images/TP_images.tsv', label='TP_images')
TP_grid = GridRegionSet(size=TP_images.size, gridsize=(32,20), label='TP_grid')

# Define some simple features from part 2
TP_fLum = LuminanceFeature(TP_grid, TP_images)
TP_fEdge = SobelEdgeFeature(TP_grid, TP_images)
TP_fCent = CentralBiasFeature(TP_grid, TP_images, measure='gaussian', sig2=0.23, nu=0.45)

# Load IKN98 feature maps and define a MapFeature
TP_saliency_maps = ImageSet('TP_saliency/TP_saliency.tsv', label='TP_saliency')
TP_fIKN = MapFeature(TP_grid, TP_saliency_maps, stat=np.mean)

TP_reward_maps = ImageSet('TP_reward/TP_reward.tsv', label='TP_reward')
TP_fReward = MapFeature(TP_grid, TP_reward_maps, stat=np.mean)

# Load fixation data
TP_fix = Fixations('TP_fixations.tsv', imageid='image_id', fixid='CURRENT_FIX_INDEX', 
                x='CURRENT_FIX_X', y='CURRENT_FIX_Y', imageset=TP_images)


Now the actual GLMM preprocessing can be performed using a _FixationModel_ object which combines fixation data, RegionSet and all features specified in the _features=_ argument into a common predictor matrix, which can then be loaded into R. With a big dataset, this could take a while, but for this tutorial, updating the model should be a matter of a few seconds. Note that the _chunks=_ argument specifies the names of columns over which data should not be aggregated, e.g. individual subjects, while the _features=_ argument contains a Python list of the actual Feature objects defined above.

In our example, this predictor matrix contains one line for each subject, image and grid cell, which will yield 8 x 15 x 48 = 5760 individual data points to be entered into GLMM. We can print the resulting model object to verify this:

In [2]:
TP_model = FixationModel(TP_fix, TP_grid, chunks=['TRIAL_INDEX', 'image_id'], features=[TP_fLum, TP_fCent, TP_fEdge, TP_fIKN, TP_fReward], dv_type='fixated')
print(TP_model)

/Users/zhangwenhu/Desktop/gridfix/gridfix/model.py:1378: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pred_new = concat([pred_new, results], ignore_index=True)


<gridfix.FixationModel, 208640 samples, DV=['fixated'], chunked by: ['TRIAL_INDEX', 'image_id']>
Fixations:
	<gridfix.Fixations data set, 2091 samples, 33 images>
Images:
	<gridfix.ImageSet "TP_images", 33 images, size=(512, 320), normalized>
Regions:
	<gridfix.GridRegionSet (TP_grid), size=(512, 320), 32x20 grid, 640 cells, memory=102400.0 kB>

Features:
	fLumin	LuminanceFeature
	fCentr	CentralBiasFeature
	fSobel	SobelEdgeFeature
	fMapFe	MapFeature
	fMapFe1	MapFeature



The resulting predictor matrix now contains one row per combination of image and region (and possible other variables used for chunking in the model specification, such as the subject id). Within each row, the column _dvFix_ indicates the fixation state of each cell, i.e., whether it was fixated (1) or not (0), while the remaining columns contain the feature values for the corresponding regions - here, mean cell luminance (fLumin) and distance from image center following an anisotropic Gaussian CB model (fCentr). 

The predictor matrix can be accessed as a DataFrame using the _predictors_ attribute of the generated model object:

In [3]:
print(TP_model.predictors)

       TRIAL_INDEX image_id regionid regionno  dvFix    fLumin    fCentr  \
0                1   575834        1        1    0.0  0.000000  0.976304   
1                1   575834        2        2    0.0  0.000000  0.969414   
2                1   575834        3        3    0.0  0.141168  0.961189   
3                1   575834        4        4    0.0  0.706211  0.951584   
4                1   575834        5        5    0.0  0.456245  0.940623   
...            ...      ...      ...      ...    ...       ...       ...   
208635         331    37367      636      636    0.0  0.108371  0.942654   
208636         331    37367      637      637    0.0  0.101557  0.953290   
208637         331    37367      638      638    0.0  0.030630  0.962596   
208638         331    37367      639      639    0.0  0.000000  0.970555   
208639         331    37367      640      640    0.0  0.000000  0.977212   

          fSobel    fMapFe   fMapFe1  
0       0.000000  0.062745  0.001961  
1       0

To facilitate analysis of the generated predictor matrix in R, GridFix also generates R source code that contains the necessary commands to import the generated data file, define factors, standardize predictors and set up a model formula. This can serve as a starting point for analysis but should be adapted to the individual factor structure of each hypothesis and dataset - as a reminder, the actual call to _glmer_ is commented out. 

The _model.save()_ function saves both the predictor matrix and the R code to files to be read into R. Note that the file name argument should be the _base name_ of generated files, e.g., model.save("tutorial") will generate tutorial.csv and tutorial.R.

In [4]:
# Print the source code here as an example
print(TP_model.r_source())

TP_model.save('TP_model')

# GridFix GLMM R source, generated on 27.05.24, 02:13:01
# 
# Predictor file:	gridfix.csv
# Fixations file:	TP_fixations.tsv
# RegionSet:		<gridfix.GridRegionSet (TP_grid), size=(512, 320), 32x20 grid, 640 cells, memory=102400.0 kB>
# DV type(s):		['fixated']

library(lme4)

gridfixdata  <- read.table("gridfix.csv", header=T, sep="\t", row.names=NULL)

# Define R factors for all chunking variables and group dummy codes
gridfixdata$TRIAL_INDEX <- as.factor(gridfixdata$TRIAL_INDEX)
gridfixdata$image_id <- as.factor(gridfixdata$image_id)

# Center and scale predictors
gridfixdata$fLumin_C <- scale(gridfixdata$fLumin, center=TRUE, scale=TRUE)
gridfixdata$fCentr_C <- scale(gridfixdata$fCentr, center=TRUE, scale=TRUE)
gridfixdata$fSobel_C <- scale(gridfixdata$fSobel, center=TRUE, scale=TRUE)
gridfixdata$fMapFe_C <- scale(gridfixdata$fMapFe, center=TRUE, scale=TRUE)
gridfixdata$fMapFe1_C <- scale(gridfixdata$fMapFe1, center=TRUE, scale=TRUE)

# NOTE: this source code can only serve as a scaff